In [1]:
import ee
import geemap
import os
import csv
import pandas as pd
import math

In [2]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [ ]:
def getNDVI(img):
    NDVI = img.normalizedDifference(['B8','B4']).rename("NDVI")
    return img.addBands(NDVI)


# collection d'image
S2_1c = ee.ImageCollection("COPERNICUS/S2").map(getNDVI)

S2_2a = ee.ImageCollection("COPERNICUS/S2_SR").map(getNDVI)

MODIS = ee.ImageCollection("MODIS/006/MOD13Q1").select('NDVI')

L5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_8DAY_NDVI').select('NDVI')

L7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_8DAY_NDVI').select('NDVI')

L8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_8DAY_NDVI').select('NDVI')

sat = [S2_1c,S2_2a,MODIS,L5,L7,L8]
nsat = ['S2_1c','S2_2a','MODIS','L5','L7','L8']

In [ ]:
table = ee.FeatureCollection("users/NelsonFernandes/VDC_NLSS3_corWGS84")

In [ ]:
taille = table.size()
t1 = table.size().getInfo()
liste = table.toList(taille).getInfo()
bb = ee.List(table.aggregate_array('DDVVVCBS11')).getInfo()

In [ ]:
stat = ['mean', 'minimum', 'quart', 'mediane', 'tquart', 'maximum']

In [ ]:
t2 = len(sat)
t3 = len(stat)
t4 = t1*t2*t3
t4

In [ ]:
def coupe (image):
    return image.clip(polygone.geometry())


def pixel_count (image):
    a = image.reduceRegion(
        reducer = ee.Reducer.count(),
        geometry = polygone.geometry(),
        scale = 30,
    )
    return ee.Image(image.setMulti(a))


def percentage (image):
    qa = image.select('QA60')
                # Bits 10 et 11 sont les nuages et les cirrus
    cloudBitMask = int(2**10)
    cirrusBitMask = int(2**11)

                # on sélectionne les pixels ayant un nuage
    masked = qa.bitwiseAnd(cloudBitMask).eq(0).And(\
                qa.bitwiseAnd(cirrusBitMask).eq(0))

    mask = image.updateMask(masked).divide(10000)
  
                # calcule du nombre de pixels avec et sans le masque de nuage
    a = mask.reduceRegion(reducer= ee.Reducer.count(),geometry= polygone.geometry())
    
    s = image.reduceRegion(reducer= ee.Reducer.count(),geometry= polygone.geometry())
  
                # calcule du pourcentage de nuage
    numa = a.getNumber('B1')
    nums = s.getNumber('B1')
    per = (nums.subtract(numa)).divide(nums).multiply(100)
  
                # ajout du pourcentage à la collection de base
    return image.set('CLOUD_PERCENTAGE_CLIP',per)

In [ ]:
# définition des statistiques

def mean (image):
    dic = image.reduceRegion(reducer = ee.Reducer.mean(),geometry= polygone.geometry(),scale= pixel)
    return ee.Image(image.setMulti(dic))

def minimum (image):
    dic = image.reduceRegion(reducer = ee.Reducer.min(),geometry= polygone.geometry(),scale= pixel)
    return ee.Image(image.setMulti(dic))

def quart (image):
    dic = image.reduceRegion(reducer = ee.Reducer.percentile([25]), geometry= polygone.geometry(),scale= pixel)
    return ee.Image(image.setMulti(dic))

def mediane (image):
    dic = image.reduceRegion(reducer = ee.Reducer.median(), geometry= polygone.geometry(),scale= pixel)
    return ee.Image(image.setMulti(dic))

def tquart (image):
    dic = image.reduceRegion(reducer = ee.Reducer.percentile([75]), geometry= polygone.geometry(),scale= pixel)
    return ee.Image(image.setMulti(dic))

def maximum (image):
    dic = image.reduceRegion(reducer = ee.Reducer.max(),geometry= polygone.geometry(),scale= pixel)
    return ee.Image(image.setMulti(dic))


In [ ]:
a=0
z=0
for i in liste:
    b=0
    polygone = ee.Feature(liste[a])
    namepoly = str(bb[a])
    
    for j in sat:
        collec = sat[b]
        namesat = str(nsat[b])
        c = 0
        
        if namesat == 'L5' or namesat == 'L7' or namesat == 'L8':
            
            precollection = collec.map(coupe).map(pixel_count)
            st = precollection.aggregate_max('NDVI')
            collection = precollection.filter(ee.Filter.eq('NDVI', st))
            pixel = 30
        
        elif namesat == 'S2_1c' or namesat == 'S2_2a':
            
            precollection = collec.filterBounds(polygone.geometry()).map(coupe).map(percentage).filter(ee.Filter.lt('CLOUD_PERCENTAGE_CLIP', 20))
            collection = precollection.select('NDVI')
            pixel = 10
        
        else:
            #cas Modis
            collection = collec.map(coupe)
            pixel = 250
            
        for y in stat:
            
            namestat = str(stat[c])
            parent = "C:/Users/nelsf/Documents/Master/VDM"
            path = os.path.join(parent, namepoly, namesat, namestat)
            
            try: 
                os.makedirs(path, exist_ok = True)  
            except OSError as error: 
                print("error")
            
            os.chdir(path)
            
            if namestat == 'mean':
                imageCollection = collection.map(mean)
            
            elif namestat == 'minimum':
                imageCollection = collection.map(minimum)
            
            elif namestat == 'quart':
                imageCollection = collection.map(quart)
            
            elif namestat == 'mediane':
                imageCollection = collection.map(mediane)
            
            elif namestat == 'tquart':
                imageCollection = collection.map(tquart)
            
            else:
                # cas namestat = 'max'
                imageCollection = collection.map(maximum)
            
            ndvi1 = ee.List(imageCollection.aggregate_array('NDVI')).getInfo()
            date1 = ee.List(imageCollection.aggregate_array('system:index')).getInfo()
            ndvi = ['NDVI'] + ndvi1
            date = ['DATE'] + date1
            
            dico = [date,ndvi]
            tab = [list(i) for i in zip(*dico)]
            
            fichier = namepoly + '_' + namesat + '_' + namestat +'.csv'
            
            df = pd.DataFrame(tab)
            df.to_csv(fichier,index=False, header=False)
            
            c += 1
            z += 1
            print(z/t4)
        b += 1
    a += 1